In [ ]:
# need to installl LLaVA https://github.com/haotian-liu/LLaVA

In [1]:
from PIL import Image
import gc

def image_parser(image_file):
    out = image_file.split(",")
    return out

def load_image(image_file):
    if image_file.startswith("http") or image_file.startswith("https"):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")
    image.resize((300, 300))
    return image

def load_images(image_files):
    out = []
    for image_file in image_files:
        image = load_image(image_file)
        out.append(image)
    return out

In [2]:
# need to do this because my GPU memory is small
if 'pipe' in locals() or 'pipe' in globals():
    print('deleting pipe')
    del pipe
    gc.collect()
    torch.cuda.empty_cache()

if 'model' in locals() or 'model' in globals():
    print('deleting model')
    del model
    gc.collect()
    torch.cuda.empty_cache()    

In [ ]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.utils import disable_torch_init
disable_torch_init()

# the code below only works for this model
model_path = "liuhaotian/llava-v1.5-7b"
model_base = None
model_name = get_model_name_from_path(model_path)

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path, model_base, model_name, offload_folder="offload", load_4bit=True
)
print(model.device)
print(model.dtype)
print('all loaded')

/home/dwangz/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/dwangz/.venv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [27]:
prompt = "<image> Describe a blend of both images."
image_file = "/home/dwangz/Downloads/calder.png,/home/dwangz/Downloads/blue_orange_green.png"
image_file = "examples/calder.png,examples.blue_orange_green.png"

from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
)
qs = prompt
qs = DEFAULT_IMAGE_TOKEN + "\n" + qs
conv_mode = "llava_v1"

from llava.conversation import conv_templates
conv = conv_templates[conv_mode].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()
print(prompt)

from llava.mm_utils import (
    process_images,
    tokenizer_image_token,
)

import torch
image_files = image_parser(image_file)
images = load_images(image_files)
for image in images:
    image.show()
image_sizes = [x.size for x in images]
images_tensor = process_images(
    images,
    image_processor,
    model.config
).to(model.device, dtype=torch.float16)

input_ids = (
    tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
    .unsqueeze(0)
    .cuda()
)

args = type('Args', (), {
    "temperature": 0,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512,
})()

with torch.inference_mode():
    print('generating...')
    output_ids = model.generate(
        input_ids,
        images=images_tensor,
        image_sizes=image_sizes,
        do_sample=True if args.temperature > 0 else False,
        temperature=args.temperature,
        top_p=args.top_p,
        num_beams=args.num_beams,
        max_new_tokens=args.max_new_tokens,
        use_cache=True,
    )

outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(outputs)

A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: <image>
<image> Describe a blend of both images. ASSISTANT:
generating...


/home/dwangz/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/dwangz/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The image features a unique artwork that combines elements of a painting and a sculpture. The painting consists of a series of circles, with one large circle in the center and three smaller circles surrounding it. The sculpture is represented by a hanging mobile, which is a three-dimensional structure made of metal, featuring a combination of red, blue, and yellow colors. The mobile is suspended from a string, creating a visually striking and creative piece of art. The painting and the mobile together create an interesting and captivating visual experience.


In [28]:
# need to do this because my GPU memory is small
if 'pipe' in locals() or 'pipe' in globals():
    print('deleting pipe')
    del pipe
    gc.collect()
    torch.cuda.empty_cache()

if 'model' in locals() or 'model' in globals():
    print('deleting model')
    del model
    gc.collect()
    torch.cuda.empty_cache()    

deleting model


In [29]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float32)
print('pipe created')
pipe.to('cuda')
print(pipe.device)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

pipe created
cuda:0


In [30]:
image = pipe(outputs).images[0]
image.show()

Token indices sequence length is longer than the specified maximum sequence length for this model (106 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['suspended from a string, creating a visually striking and creative piece of art. the painting and the mobile together create an interesting and captivating visual experience.']


  0%|          | 0/50 [00:00<?, ?it/s]